In [38]:
import overpy
import geopy
api = overpy.Overpass()


In [10]:
def make_query(lat,lon,radius=1000):
    return """<query type="way">
    <around lat=" """ + str(lat) + """ " lon=" """ + str(lon) + """ " radius=" """ + str(radius) + """ "/>
    </query>
    <union>
      <item/>
      <recurse type="down"/>
    </union>
    <print/>"""

In [11]:
make_query(33.255202,-116.258510)

'<query type="way">\n    <around lat=" 33.255202 " lon=" -116.25851 " radius=" 1000 "/>\n    </query>\n    <union>\n      <item/>\n      <recurse type="down"/>\n    </union>\n    <print/>'

In [13]:
# fetch all ways and nodes
example_query = """
    way(50.746,7.154,50.748,7.157) ["highway"];
    (._;>;);
    out body;
    """

test_query = """<query type="way">
    <around lat="33.255202" lon="-116.258510" radius="1000"/>
</query>
<union>
  <item/>
  <recurse type="down"/>
</union>
<print/>"""

result = api.query(make_query(33.255202,-116.258510))

print(len(result.ways))
for way in result.ways:
    
    #print("Name: %s" % way.tags.get("name", "n/a"))
    print("  Highway: %s" % way.tags.get("highway", "n/a"))
    #print("  Nodes:")
    #for node in way.nodes:
    #    print("    Lat: %f, Lon: %f" % (node.lat, node.lon))

8
  Highway: track
  Highway: path
  Highway: n/a
  Highway: n/a
  Highway: n/a
  Highway: n/a
  Highway: n/a
  Highway: track


In [52]:
from geopy.distance import distance
def sq_dist(latlon1,latlon2):
    return (latlon1[0]-latlon2[0])**2 + (latlon1[1]-latlon2[1])**2 
def nearest_node(latlon,way):
    dists = [distance(latlon,latlon_float((node.lat,node.lon))).miles for node in way.nodes]
    idx = np.argmin(dists)
    return (lambda n: latlon_float((n.lat,n.lon)))(way.nodes[idx]), np.min(dists)
def latlon_float(latlon):
    return (float(latlon[0]),float(latlon[1]))

#consider removing motorway, residential
def is_valid_highway(label):
    valid_highways = 'motorway,trunk,primary,secondary,tertiary,unclassified,residential'.split(',')
    return np.any([label == l for l in valid_highways]) 

def closest_valid_node(latlon,ways):
    highway_labels = [way.tags.get("highway", "n/a") for way in result.ways]
    is_valid = list(map(is_valid_highway,highway_labels))
    valid_ways = [way for way,v in zip(result.ways,is_valid) if v]
    if len(valid_ways) == 0:
        return (np.nan,np.nan), np.nan
    coords, dists = zip(*[nearest_node(latlon,way) for way in valid_ways])
    min_idx = np.argmin([dists])
    min_dist = dists[min_idx]
    min_coord = coords[min_idx]
    return min_coord, min_dist

In [55]:
latlon = (33.468705, -117.020323)
result = api.query(make_query(*latlon))
print(len(result.ways))
print(closest_valid_node(latlon,result.ways))


25
((33.4714886, -117.018138), 0.22963041597937678)


In [46]:
highway_labels = [way.tags.get("highway", "n/a") for way in result.ways]
is_valid = list(map(is_valid_highway,highway_labels))
valid_ways = [way for way,v in zip(result.ways,is_valid) if v]
coords, dists = zip(*[nearest_node(latlon,way) for way in valid_ways])
min_idx = np.argmin([dists])
min_dist = dists[min_idx]
min_coord = coords[min_idx]

In [48]:
min_coord

(32.729824, -116.866913)